In [77]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import os
from models.binarized_modules import binarized
import math
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
# from binarized_modules import  BinarizeLinear,BinarizeConv2d

In [78]:
import models.cim_fc
import models.cim_conv

In [79]:
import importlib

In [80]:
importlib.reload(models.cim_conv)
importlib.reload(models.cim_fc)

<module 'models.cim_fc' from '/shares/bulk/earapidis/dev/BinarizedNN/models/cim_fc.py'>

In [81]:
cuda = False

In [82]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [83]:
test_batch_size=1

In [84]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [85]:
from models.mlp import MLP,MLP_CIM
model_idx = 2
models_path = os.path.abspath(f"/shares/bulk/earapidis/dev/BinarizedNN/saved_models/mlp/model_{model_idx}")
model_path = os.path.join(models_path,f"best.pth")

model = MLP()
model.load_state_dict(torch.load(model_path))
if cuda:
    torch.cuda.set_device(0)
    model.cuda()

In [86]:
# from models.lenet_5 import BinarizedLeNet5_BN as Lenet_5
# from models.lenet_5 import BinarizedLeNet5_BN_CIM as Lenet_5_CIM

# model_idx = 1
# models_path = os.path.abspath(f"/shares/bulk/earapidis/dev/BinarizedNN/saved_models/lenet_5/model_{model_idx}")
# model_path = os.path.join(models_path,f"best.pth")
# model = Lenet_5()
# model.load_state_dict(torch.load(model_path))
# if cuda:
#     torch.cuda.set_device(0)
#     model.cuda()

In [87]:
model.eval()

MLP(
  (fc1): BinarizeLinear(in_features=784, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh1): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc2): BinarizeLinear(in_features=512, out_features=32, bias=True)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh2): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc3): BinarizeLinear(in_features=32, out_features=10, bias=True)
  (drop): Dropout(p=0.1, inplace=False)
)

In [88]:
images, labels = next(iter(test_loader))
print(f"image shape: {images.shape}")

image shape: torch.Size([1, 1, 28, 28])


In [89]:
x = images.view(-1, 28*28)
x = x.detach()

In [90]:

# x=model.conv1(images)
# x=model.bn1(x)
# x=model.htanh1(x)
# x=model.pool1(x)
# x = x.detach()

In [91]:
inputs = x
inputs = binarized(inputs)
inputs.shape

torch.Size([1, 784])

In [92]:
filters = model.fc1.weight.data
# filters = model.conv2.weight.data
filters = binarized(filters)
filters.shape

torch.Size([512, 784])

In [93]:
# ref = model.conv2(x)
ref = model.fc1(x)
ref.shape

torch.Size([1, 512])

In [94]:
mapping = False
# mapping = True
# model_cim = Lenet_5_CIM(Num_rows=32, Num_Columns=32, mode="ideal",workers=16,transient=False,checkboard=mapping,mapping=mapping)
model_cim = MLP_CIM(Num_rows=32, Num_Columns=32, mode="ideal",workers=16,transient=False,checkboard=mapping,mapping=mapping)
model_cim.load_state_dict(model.state_dict())

model_cim.eval()

MLP_CIM(
  (fc1): BinarizeLinearInference(in_features=784, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh1): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc2): BinarizeLinearInference(in_features=512, out_features=32, bias=True)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh2): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc3): BinarizeLinearInference(in_features=32, out_features=10, bias=True)
  (drop): Dropout(p=0.1, inplace=False)
)

In [95]:
output = model_cim.fc1(inputs)
# output = model_cim.conv2(inputs)
torch.equal(ref,output)

True